In [9]:
import pandas as pd
import requests
from urllib.parse import urlencode
import math

In [10]:
def openaqv2_measurements(
        date_from = '2000-01-01',
        date_to = '2022-09-29',
        location_id = None,
        sort = None,
        page = 1,
        entries = 0
):
    query_url = 'https://api.openaq.org/v2/measurements'
    params = {
        'date_from' : date_from,
        'date_to' : date_to,
        'location_id' : location_id,
        'sort' : 'desc',
        'order_by' : 'datetime',
        'page' : 1
    }

    api_params = {x : y for (x,y) in params.items() if y != None}
    url = f"{query_url}?{urlencode(api_params)}"

    session = requests.Session()
    response = session.get(url)

    columns=['location_id', 'location', 'parameter', 'value', 'date_utc', 'unit', 'latitude', 'longitude', 'country', 'city', 'isMobile', 'isAnalysis', 'entity', 'sensorType']
    df = pd.DataFrame(columns=columns)

    for result in response.json()['results']:
        vals = pd.Series([
            result['locationId'],
            result['location'],
            result['parameter'],
            result['value'],
            result['date']['utc'],
            result['unit'],
            result['coordinates']['latitude'],
            result['coordinates']['longitude'],
            result['country'],
            result['city'],
            result['isMobile'],
            result['isAnalysis'],
            result['entity'],
            result['sensorType']
        ])

        new_row = pd.Series({x : y for (x,y) in zip(columns, vals)})
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

    if page == 1 and entries != 0:
        found = response.json()["meta"]["found"]
        pages = (math.ceil(entries / 100)) if found >= entries else (math.ceil(found / 100))
        
        for i in range(2, pages + 1):
            df = pd.concat([df, openaqv2_measurements(date_from, date_to, location_id, sort, i)], ignore_index=True)

        if df.shape[0] > entries:
            df = df.loc[:entries - 1, :]
                   
    session.close()
    return df

In [11]:
test = openaqv2_measurements(entries=130)
test

KeyError: 'results'